|Name|ID|Kaggle User Name|
|------------|------- |------- |
|Johney Jijin|18032679|@jijinsas
|Jamie Scott |18041508|@jamiescott|

## Google Analytics Customer Revenue Prediction<a class="tocSkip">
## 158.333 Assignment 3 - S2, 2019- Part 2<a class="tocSkip">

## Modelling using XGBoost and LGBM Classifier and Regressor

__Initially,our models trained on the Random Forest regressor had given us slight or very minute improvement. As a result, we used the approach of training models using XGBoost,LGBM Classifier and LGBM Regressor.The intial approach was only a one step approach where two of the xgb and lgbm regressor were simply trained after engineering features and using feature score.The top 100 features were picked from the lowest RMSE to the highest.This resulted in a quite a significant improvent in accuracy.We set LGB as our baseline model as it gave us best the accuracy.We then added a step before regressingp, which meant to have a two step approach.The two step approach involved a getting classification probabilities by training a LGB classifier on the whole train set along with features scored on the whole set. Then adding those probabilites to the train and test set. We then trained a reduced train set which included only the Purchase made fullVisitorId.The predictions from this model was multiplied with the test probabilities generated earlier.This gave us much better model.__

In [43]:
%run prediction.py
train = pd.read_csv('/home/jijin/Documents/158333/Assignment_3/trainDummies2.csv', infer_datetime_format=True, parse_dates=['date'], 
                      dtype={'fullVisitorId': 'str'})

<IPython.core.display.Javascript object>

### Adding engineeered features on the train set

In [44]:
%run prediction.py
train=add_features(train)

<IPython.core.display.Javascript object>

In [45]:
%run prediction.py
test= pd.read_csv('/home/jijin/Documents/158333/Assignment_3/testDummies.csv', infer_datetime_format=True, parse_dates=['date'], 
                      dtype={'fullVisitorId': 'str'})

<IPython.core.display.Javascript object>

### Adding engineeered features on the test set

In [46]:
test=add_features(test)

<IPython.core.display.Javascript object>

In [47]:
%run prediction.py
print(train.shape)
print(test.shape)

(1708337, 422)
(401589, 422)


<IPython.core.display.Javascript object>

In [48]:
exclude_cols=['date','fullVisitorId','visitId','totals_transactionRevenue','trafficSource_adContent', 
              'trafficSource_adwordsClickInfo.adNetworkType','logRevenue', 'wasPurchaseMade','totals_totalTransactionRevenue',
              ]
cat_cols=['channelGrouping','customDimensions','device_browser','device_deviceCategory','device_operatingSystem',
          'geoNetwork_city','geoNetwork_continent','geoNetwork_subContinent','geoNetwork_country', 
          'geoNetwork_metro','geoNetwork_networkDomain','geoNetwork_region','trafficSource_campaign',
          'trafficSource_keyword','trafficSource_medium', 'trafficSource_referralPath', 'trafficSource_source',
          'weekofyear','Hour','Day','Weekday', 'Week', 'Month']

num_cols=[x for x in train.columns if (train[x].dtype == np.float64  or train[x].dtype == np.int64 or train[x].dtype == np.uint8 )]
num_cols=[x for x in num_cols if (x not in exclude_cols and x not in cat_cols)]
num_cols=[x for x in num_cols if train[x].isna().sum()/len(train[x])<0.1]

In [80]:
colors = ['lightslategray',] * 2
colors[1] = 'crimson'
x=["Purchase Not Made","Purchase Made"]
y=train.wasPurchaseMade.value_counts()
fig = go.Figure([go.Bar(x=x, y=y,marker_color=colors,text=y,name='Primary Product',
            textposition='auto', )])

fig.update_layout(title_text='Purchase made vs No Purchase made Balance',xaxis_tickfont_size=14,yaxis=dict(title='Purchase Count',titlefont_size=16,tickfont_size=14),height=800)
fig.show()

### Creating X and y variables, one set of X and y based on the whole set and the second set based on just Purchase made

In [87]:
X=train[num_cols].fillna(0)
y=np.log1p(train['totals_transactionRevenue'])

(1708337, 384)
(1708337,)
(401589, 384)


<IPython.core.display.Javascript object>

In [107]:
X_purchaseMade=train[num_cols][train.wasPurchaseMade==1]
y_purchaseMade=np.log1p(train['totals_transactionRevenue'][train.wasPurchaseMade==1])

## Feature Scoring

__Feature Scored on whole set and also on Purchase made sample for Model training.__

In [108]:
%%notify
%run prediction.py
#scores=feature_score(X,y)
scores_purchaseMade=feature_score(X_purchaseMade,y_purchaseMade)
scores.head(10)

,RMSE
Feature,
totals_pageviews,1.706428
totals_hits,1.719384
totals_sessionQualityDim,1.747544
totals_timeOnSite,1.776740
Mean_Pageviews_by_networkDomain_roll12,1.824121
Sum_Pageviews_by_networkDomain_roll12,1.824122
referral_short,1.825694
Mean_Pageviews_by_networkDomain_roll6,1.827275
Sum_Pageviews_by_networkDomain_roll6,1.827275


<IPython.core.display.Javascript object>

In [109]:
scores_purchaseMade.head(10)

,RMSE
Feature,
totals_hits,1.444587
totals_pageviews,1.450909
totals_timeOnSite,1.459137
desktop,1.459722
device_isMobile,1.459774
mobile,1.460317
totals_newVisits,1.460959
visitNumber,1.462748
trafficSource_isTrueDirect,1.464796


### Using the first 100 features

In [113]:
scored_features_PM=scores_purchaseMade.index[:100]
scored_features=scores.index[:100]

In [192]:
%run prediction.py
xgb_scores=[]
lgb_scores=[]
X_train, X_test, y_train, y_test = train_test_split(X_purchaseMade[scored_features_PM],y_purchaseMade, test_size= 0.2, random_state=123)
xgb_scores.append(xgb_cross_val(X_train, y_train,X_test,y_test))
lgb_scores.append(lgb_cross_val(X_train, y_train,X_test,y_test))
print(xgb_scores)
print(lgb_scores)

[(1.36, 0.11)]
[(1.36, 0.1)]


## Two Steps Process using LGBM Classifier and Regressor

### Training whole set on LGB classifier and features scores on whole set.

In [90]:
%run prediction.py
positives=(train['totals_transactionRevenue'].fillna(0) > 0).astype(np.uint8)
trn_preds_whole,tst_preds_whole,clf_whole_set=run_lgbClassifier(train,test,positives,scored_features)

Features: 100
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.0264478
[100]	valid_0's binary_logloss: 0.0250068
[150]	valid_0's binary_logloss: 0.0251319
Early stopping, best iteration is:
[114]	valid_0's binary_logloss: 0.0248991
0.987662823155957
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.0273415
[100]	valid_0's binary_logloss: 0.0257588
[150]	valid_0's binary_logloss: 0.0277081
Early stopping, best iteration is:
[136]	valid_0's binary_logloss: 0.0255113
0.9880663907437313
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.0257352
[100]	valid_0's binary_logloss: 0.0241528
[150]	valid_0's binary_logloss: 0.0242118
Early stopping, best iteration is:
[132]	valid_0's binary_logloss: 0.023951
0.9887995404368292
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.0255822
[100]	valid_0's binary_logloss: 0.024045

#### Adding classification probabilities to both train and test set

In [92]:
train['Purchase_probability'] = trn_preds_whole
test['Purchase_probability'] = tst_preds_whole

### Training and predicting on LGB Regressor on smaller sample of Purchase Made

In [114]:
%run prediction.py
#y=train['totals_transactionRevenue'].fillna(0)
predictions_PurchaseMade=run_lgbRegressor(X_purchaseMade,y_purchaseMade,test,scored_features_PM)

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.326843
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[90]	valid_0's rmse: 0.306942
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 0.283214
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[87]	valid_0's rmse: 0.180243
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[88]	valid_0's rmse: 0.128911


In [138]:
#del sub_lgb
sub_lgb = pd.DataFrame()
sub_lgb['fullVisitorId'] = test.fullVisitorId
sub_lgb['PredictedLogRevenue'] = (predictions_PurchaseMade)*test.Purchase_probability
sub_lgb=pd.DataFrame(np.log1p(sub_lgb.groupby('fullVisitorId')['PredictedLogRevenue'].sum()))
sub_lgb=sub_lgb.reset_index()

In [132]:
sub_lgb1 = pd.DataFrame()
sub_lgb1['fullVisitorId'] = test.fullVisitorId
sub_lgb1['PredictedLogRevenue'] = (predictions_PurchaseMade)*test.Purchase_probability
sub_lgb1=pd.DataFrame(sub_lgb1.groupby('fullVisitorId')['PredictedLogRevenue'].mean())
sub_lgb1=sub_lgb1.reset_index()

In [173]:
sub_lgb.head(10)

,fullVisitorId,PredictedLogRevenue
0,0000018966949534117,0.016921
1,0000039738481224681,0.001262
2,0000073585230191399,0.001204
3,0000087588448856385,0.001255
4,0000149787903119437,0.001246
5,0000196310838896290,0.827706
6,00001995526696366,0.001494
7,0000255704530917106,0.053541
8,0000268499301061358,0.001219
9,0000276747989270229,0.001254


In [ ]:
sub_lgb.to_csv('LGB_PROBABILITY_With_Predictions02.csv',index=False)

In [141]:
sub_lgb.to_csv('LGB_PROBABILITY_With_Predictions03.csv',index=False)

In [143]:
from kaggle.api.kaggle_api_extended import KaggleApi 

# from kaggle.api.kaggle_api_extended import KaggleApi 
api = KaggleApi()


In [144]:
import sys,os,os.path #for kaggle api set-up

os.environ['KAGGLE_USERNAME'] = "jijinsas"
os.environ['KAGGLE_KEY'] = "3189658b2791edd31867b59fc52f6c60"

## Submissions

__Our final model therefore utilises the LGBM Classifier to produce a range of probabilities that a web visit results in transaction. We train a LGBM regressor on only the values in the training data that correspond to actual transactions.__

__The regression on this particular subset will naturally over estimate the size of transaction for those visits that don’t result in a purchase but the probabilities we’ve previously created will counter that but reducing their predicted amount when we multiply the two together (due to a low probability).__


In [191]:
submissionlist = !kaggle competitions submissions -c ga-customer-revenue-prediction
filename = []
date = []
desc = []
status = []
private = []
for i, row in enumerate(submissionlist):
    if i>1:
        filename.append(submissionlist[i][0:41])
        date.append(submissionlist[i][44:63])
        desc.append(submissionlist[i][64:69])
        status.append(submissionlist[i][78:86])
        private.append(submissionlist[i][95:110])
                
scores = {'Filename':filename, 'Date': date, 'Description': desc, 'Status': status, 'Score': private}
scores_df = pd.DataFrame(scores)
scores_df['Date'] = pd.to_datetime(scores_df['Date'])
scores_df = scores_df[scores_df['Status']=='complete']
scores_df['Score']=scores_df['Score'].astype(float)
scores_df.drop('Status', axis=1, inplace = True)
scores_df.reset_index(drop=True, inplace=True)

scores_df

,Filename,Date,Description,Score
0,LGB_PROBABILITY_With_Predictions05.csv,2019-10-18 01:27:49,None,1.51903
1,LGB_PROBABILITY_With_Predictions04.csv,2019-10-18 01:26:16,None,0.93467
2,LGB_PROBABILITY_With_Predictions03.csv.cs,2019-10-18 00:29:46,None,0.88603
3,LGB_PROBABILITY_With_Predictions02.csv,2019-10-18 00:16:21,None,0.93467
4,LGB_PROBABILITY_With_Predictions01.csv,2019-10-18 00:10:33,None,1.02205
5,LGB_PROBABILITY_With_Predictions.csv,2019-10-17 23:36:36,None,0.93467
6,lgb_submission_33_withclassifier.csv,2019-10-17 10:26:02,None,2.12111
7,lgb_submission_32_withclassifier.csv,2019-10-17 10:02:12,None,1.10435
8,lgb_submission_31_withclassifier.csv,2019-10-17 10:01:07,None,1.10435
9,lgb_submission_30_withclassifier.csv,2019-10-17 09:47:17,None,1.10504


In [166]:
colors = ['lightslategray',] * 5
colors[4] = 'green'
x=["Random Forest-First Submission","Best Random Forest","Best XGB","Best LGB without classification","Best LGB with classification"]
y=[18.7,11.2,1.62,1.05,0.886]
fig = go.Figure([go.Bar(x=x, y=y,marker_color=colors,text=y,name='Primary Product',
            textposition='auto', )])

fig.update_layout(title_text='Road to better accuracy',xaxis_tickfont_size=14,yaxis=dict(title='Accuracy',titlefont_size=16,tickfont_size=14),height=800)
fig.show()

![BEST_SUBMISSION](best_submission.PNG "Best Submission")

## Conclusion
__In tackling the challenge presented by this Kaggle competition and assignment, we were able to employ a number of strategies to handle big(ish) data on our local machines and formulate a successful multi-stage approach to predicting customer spending at this particular Google store with a relatively low root mean square error of 0.886.__